<a id="top"></a>
# 04 · Resumen y visualización de resultados

**Qué hace este notebook**  
Inspecciona los **resultados de entrenamientos continual** y presenta:
- Un **resumen por experimento** y otro **por tarea** (historiales).
- Lectura de **telemetría de emisiones** de *CodeCarbon* (`emissions.csv`) y del log ligero `telemetry.jsonl`.
- Un **CSV enriquecido** con rendimiento + emisiones para análisis posterior.
- Gráficos rápidos: MAE, olvido relativo y emisiones (kg CO₂e), además de un *scatter* rendimiento vs. emisiones.

**Fuentes de datos**
- `outputs/continual_*/*/manifest.json` (o `metrics.json`) → historiales por tarea.
- `outputs/continual_*/continual_results.json` → métricas a nivel de experimento.
- `outputs/continual_*/emissions.csv` (si *CodeCarbon* estaba activo en el run).
- `outputs/continual_*/telemetry.jsonl` (eventos del runner).

## ✅ Prerrequisitos
- Haber ejecutado **03_TRAIN_CONTINUAL.ipynb** (para poblar `outputs/continual_*`).
- (Opcional) Tener *CodeCarbon* instalado/activado para disponer de `emissions.csv`.

---

<a id="toc"></a>
## Índice

- [1) Imports y rutas base](#sec-01)  
- [2) Utilidades de parseo y lectura robusta](#sec-02)  
- [3) Resumen “continual” (por run)](#sec-03)  
- [4) Resumen de entrenamiento por tarea (manifest/metrics)](#sec-04)  
- [5) Gráficos rápidos](#sec-05)  
- [6) Inspección de un experimento concreto](#sec-06)
- [7) Inspección detallada: CodeCarbon + Telemetry (último run)](#sec-07)  
- [8) Agregados por preset/método con emisiones](#sec-08)


<a id="sec-01"></a>
## 1) Imports y rutas base

**Objetivo**  
Configurar el entorno mínimo de lectura/visualización:
- Detectar la **raíz del repo** (`ROOT`) tanto si ejecutas desde `notebooks/` como desde la raíz.  
- Definir `OUT = ROOT / "outputs"` como carpeta base de resultados.

> No modifica ni reescribe archivos de los runs: **solo lee** y visualiza.  

[↑ Volver al índice](#toc)


In [ ]:
# Celda 0 — Config (versión dinámica)
from pathlib import Path
import pandas as pd, numpy as np, math, re, os, json
from datetime import datetime

# -------- Raíz de outputs del proyecto --------
OUT = Path(os.environ.get("OUT_DIR", "/home/cesar/proyectos/TFM_SNN/outputs")).resolve()

# -------- Filtros "paper set" (por defecto) --------
# Puedes sobreescribirlos con env vars: PRESET_KEEP, ENCODER_KEEP, SEED_KEEP, METHODS_KEEP
def _parse_set(env_name, default):
    val = os.environ.get(env_name, None)
    if val is None:
        return default
    if val.strip().lower() in {"none", "", "all"}:
        return None
    return set([v.strip() for v in val.split(",") if v.strip()])

PRESET_KEEP   = _parse_set("PRESET_KEEP",   {"std"})   # None → no filtra por preset
ENCODER_KEEP  = _parse_set("ENCODER_KEEP",  {"rate"})   # None → no filtra
SEED_KEEP     = _parse_set("SEED_KEEP",     None)       # p.ej. "42,7"
METHODS_KEEP  = _parse_set("METHODS_KEEP",  None)

ONLY_NEW_RUNNER = os.environ.get("ONLY_NEW_RUNNER", "1") not in {"0","false","False"}

# Ventana temporal: SINCE="2025-11-11" o SINCE="3d" (últimos 3 días). Si no se da, no filtra por fecha.
# SINCE = os.environ.get("SINCE", "").strip()
SINCE="2025-11-11"

def _compute_mtime_from(since_str: str):
    if not since_str:
        return None
    try:
        if since_str.lower().endswith("d"):
            days = int(since_str[:-1])
            return pd.Timestamp.now().normalize() - pd.Timedelta(days=days)
        return pd.Timestamp(since_str)
    except Exception:
        return None
MTIME_FROM = _compute_mtime_from(SINCE) or None

# -------- Comparabilidad "dura" --------
STRICT_CFG = os.environ.get("STRICT_CFG", "1") not in {"0","false","False"}

# -------- Métrica compuesta --------
ALPHA_COMPOSITE = float(os.environ.get("ALPHA_COMPOSITE", "0.5"))

# -------- Opciones de informe / narrativa --------
# IGNORE_NAIVE_IN_REPORTS = os.environ.get("IGNORE_NAIVE", "0") in {"1","true","True"}
GROUP_BY_FULL_METHOD    = os.environ.get("GROUP_BY_FULL_METHOD", "0") in {"1","true","True"}
# RELATIVE_BASELINE       = os.environ.get("RELATIVE_BASELINE", "ewc")  # "naive","ewc","auto", o "" para desactivar
# BASELINE_MATCH_STRICT   = os.environ.get("BASELINE_MATCH_STRICT", "1") not in {"0","false","False"}

IGNORE_NAIVE_IN_REPORTS = True       # excluye naive de tablas/plots de informe
RELATIVE_BASELINE = "ewc"            # baseline para comparativas
BASELINE_MATCH_STRICT = True         # emparejamiento estricto de runs comparables

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 180)

# -------- Carpeta SUMMARY dinámica --------
def _token(val):
    if val is None: return "all"
    if isinstance(val, (set, list, tuple)):
        return "-".join(sorted(map(str, val)))
    return str(val)

now = pd.Timestamp.now(tz="Europe/Madrid")
auto_label = f"paperset_{_token(PRESET_KEEP)}_{_token(ENCODER_KEEP)}_{now.strftime('%Y-%m-%d_%H%M')}"
SUMMARY_LABEL = os.environ.get("SUMMARY_LABEL", "").strip() or auto_label

SUMMARY = (OUT / "summary" / SUMMARY_LABEL).resolve()
SUMMARY.mkdir(parents=True, exist_ok=True)
print(f"[SUMMARY] {SUMMARY}")

# Symlink "latest" → esta ejecución (si el FS lo permite)
LATEST = OUT / "summary" / "latest"
try:
    if LATEST.exists() or LATEST.is_symlink():
        LATEST.unlink()
    LATEST.symlink_to(SUMMARY, target_is_directory=True)
except Exception:
    pass

# Guardar manifest de filtros/ajustes para trazabilidad
manifest = {
    "summary_label": SUMMARY_LABEL,
    "created_at": now.isoformat(),
    "OUT": str(OUT),
    "filters": {
        "PRESET_KEEP": sorted(list(PRESET_KEEP)) if PRESET_KEEP else None,
        "ENCODER_KEEP": sorted(list(ENCODER_KEEP)) if ENCODER_KEEP else None,
        "SEED_KEEP": sorted(list(SEED_KEEP)) if SEED_KEEP else None,
        "METHODS_KEEP": sorted(list(METHODS_KEEP)) if METHODS_KEEP else None,
        "ONLY_NEW_RUNNER": ONLY_NEW_RUNNER,
        "MTIME_FROM": str(MTIME_FROM) if MTIME_FROM is not None else None,
    },
    "comparability": {"STRICT_CFG": STRICT_CFG},
    "composite": {"ALPHA_COMPOSITE": ALPHA_COMPOSITE},
    "report_opts": {
        "IGNORE_NAIVE_IN_REPORTS": IGNORE_NAIVE_IN_REPORTS,
        "GROUP_BY_FULL_METHOD": GROUP_BY_FULL_METHOD,
        "RELATIVE_BASELINE": RELATIVE_BASELINE,
        "BASELINE_MATCH_STRICT": BASELINE_MATCH_STRICT,
    },
}
(SUMMARY / "summary_manifest.json").write_text(json.dumps(manifest, indent=2), encoding="utf-8")

# --- Manifest "último" para reutilizar rutas en otras celdas/notebooks
LAST_PATH = OUT / "summary" / "_last.json"

def _last_update(**kwargs):
    # Carga el estado actual (si existe)
    try:
        cur = json.loads(LAST_PATH.read_text(encoding="utf-8"))
    except Exception:
        cur = {}

    # Campos fijos útiles
    cur.update({
        "summary_label": SUMMARY_LABEL,
        "summary_dir": str(SUMMARY.resolve()),
        "created_at": now.isoformat(),
    })

    # Normaliza y añade las nuevas rutas/etiquetas
    for k, v in list(kwargs.items()):
        if isinstance(v, (str, Path)):
            kwargs[k] = str(Path(v).resolve())
    cur.update(kwargs)

    LAST_PATH.write_text(json.dumps(cur, indent=2), encoding="utf-8")
    print(f"[LAST] actualizado → {LAST_PATH}")

# Crea/actualiza _last.json al inicio con la info básica de esta ejecución
_last_update()


<a id="sec-02"></a>
## 2) Utilidades de parseo y lectura robusta

**Objetivo**  
Apoyarse en utilidades **canónicas** del proyecto (`src.utils_exp`) para:
- `safe_read_json`: lectura tolerante a errores (devuelve `{}` si no existe o está corrupto).  
- `parse_exp_name`: extraer metadatos desde el nombre de la carpeta `continual_*` (preset, método, encoder, seed, modelo…).  
- `build_runs_df` y `aggregate_and_show`: construir un **DataFrame** consolidado por experimento y, opcionalmente, generar **agregados** y guardarlos.

Se añaden *helpers* locales para:
- Encontrar el primer JSON disponible por **tarea** (`manifest.json` o `metrics.json`).  
- Leer **CodeCarbon** (`emissions.csv`) de forma **tolerante a versiones** (columnas pueden variar según versión).  
- Leer el último evento de `telemetry.jsonl` para capturar `elapsed_sec` y `emissions_kg` si el runner los registró.

[↑ Volver al índice](#toc)


In [ ]:
# Celda 1 — Utils

def _safe_float(x, default=np.nan):
    try:
        if x is None: return default
        if isinstance(x, (int, float)): return float(x)
        s = str(x).strip()
        if s.lower() in {"nan","none","null",""}: return default
        return float(s)
    except Exception:
        return default

def _read_json(p: Path):
    try:
        if p.exists():
            return json.loads(p.read_text(encoding="utf-8"))
    except Exception:
        pass
    return None

def _read_csv_df(p: Path):
    try:
        if p.exists():
            return pd.read_csv(p)
    except Exception:
        pass
    return None

def _abs_run_dir(rel: str|Path) -> Path:
    rel = str(rel)
    return (OUT / rel) if not rel.startswith(str(OUT)) else Path(rel)

def run_mtime(rel: str|Path) -> float:
    rd = _abs_run_dir(rel)
    mt = 0.0
    for root, _, files in os.walk(rd):
        for f in files:
            try:
                mt = max(mt, (Path(root)/f).stat().st_mtime)
            except Exception:
                pass
    return mt

def canonical_method(m: str) -> str:
    if not m: return "none"
    m = m.lower()
    if m.startswith("ewc"):        return "ewc"
    if m.startswith("as-snn"):     return "as-snn"
    if m.startswith("sa-snn"):     return "sa-snn"
    if m.startswith("sca-snn"):    return "sca-snn"
    if m.startswith("rehearsal"):  return "rehearsal"
    if m.startswith("naive"):      return "naive"
    return m

def _read_forgetting(run_dir: Path) -> dict:
    """
    Lee primero forgetting_summary.json (preferente).
    Si no existe, lee forgetting.json (anidado por tareas) y calcula la media.
    Devuelve claves normalizadas: circuito1_forget_abs/rel, circuito2_forget_abs/rel, avg_forget_rel.
    """
    def _nanmean2(a, b):
        vals = [x for x in [a, b] if x is not None and not math.isnan(_safe_float(x))]
        return float(np.mean([_safe_float(x) for x in vals])) if vals else np.nan

    # 1) summary (preferido)
    summ = _read_json(run_dir / "forgetting_summary.json")
    if isinstance(summ, dict):
        c1_abs = _safe_float(summ.get("circuito1_forget_abs", summ.get("task_1_forget_abs")))
        c1_rel = _safe_float(summ.get("circuito1_forget_rel", summ.get("task_1_forget_rel")))
        c2_abs = _safe_float(summ.get("circuito2_forget_abs", summ.get("task_2_forget_abs")))
        c2_rel = _safe_float(summ.get("circuito2_forget_rel", summ.get("task_2_forget_rel")))
        avg_rel = _safe_float(summ.get("avg_forget_rel", summ.get("avg_forgetting_rel")))
        if math.isnan(avg_rel):
            avg_rel = _nanmean2(c1_rel, c2_rel)
        return {
            "circuito1_forget_abs": c1_abs,
            "circuito1_forget_rel": c1_rel,
            "circuito2_forget_abs": c2_abs,
            "circuito2_forget_rel": c2_rel,
            "avg_forget_rel":       avg_rel,
        }

    # 2) forgetting.json (anidado por tareas) o plano
    js = _read_json(run_dir / "forgetting.json") or {}
    if not isinstance(js, dict):
        return {}

    def pick(d, *keys):
        for k in keys:
            if k in d: return d[k]
        return None

    c1_abs = _safe_float(pick(js, "circuito1_forget_abs","task_1_circuito1_forget_abs","c1_forget_abs","task_1_forget_abs"))
    c1_rel = _safe_float(pick(js, "circuito1_forget_rel","task_1_circuito1_forget_rel","c1_forget_rel","task_1_forget_rel"))
    c2_abs = _safe_float(pick(js, "circuito2_forget_abs","task_2_circuito2_forget_abs","c2_forget_abs","task_2_forget_abs"))
    c2_rel = _safe_float(pick(js, "circuito2_forget_rel","task_2_circuito2_forget_rel","c2_forget_rel","task_2_forget_rel"))
    avg_rel = _safe_float(pick(js, "avg_forget_rel","avg_forgetting_rel","mean_forget_rel","forget_rel_avg"))

    # Si sigue faltando, leer anidado por tarea
    if ("circuito1" in js) and isinstance(js["circuito1"], dict):
        if math.isnan(c1_abs): c1_abs = _safe_float(js["circuito1"].get("forget_abs"))
        if math.isnan(c1_rel): c1_rel = _safe_float(js["circuito1"].get("forget_rel"))
    if ("circuito2" in js) and isinstance(js["circuito2"], dict):
        if math.isnan(c2_abs): c2_abs = _safe_float(js["circuito2"].get("forget_abs"))
        if math.isnan(c2_rel): c2_rel = _safe_float(js["circuito2"].get("forget_rel"))

    if math.isnan(avg_rel):
        avg_rel = _nanmean2(c1_rel, c2_rel)

    return {
        "circuito1_forget_abs": c1_abs,
        "circuito1_forget_rel": c1_rel,
        "circuito2_forget_abs": c2_abs,
        "circuito2_forget_rel": c2_rel,
        "avg_forget_rel":       avg_rel,
    }


<a id="sec-03"></a>
## 3) Resumen “continual” + emisiones (por run)

**Objetivo**  
Construir un **resumen a nivel de experimento** a partir de `continual_results.json` y **enriquecerlo** con:
- **Emisiones** de *CodeCarbon* (`emissions.csv`): `emissions_kg`, `energy_kwh`, `cpu_kwh`, `gpu_kwh`, `ram_kwh`, `duration_s`.  
- **Telemetría del runner** (`telemetry.jsonl`): `telemetry_elapsed_sec`, `telemetry_emissions_kg` (si existen).

**Salida adicional**
- Se guarda `outputs/summary/runs_with_emissions.csv` (rendimiento + emisiones) listo para graficar o compartir.

> Si no hay `emissions.csv` (no activaste CodeCarbon), las columnas de emisiones quedarán **vacías** para esos runs.

[↑ Volver al índice](#toc)


In [ ]:
# Celda 2 — Reconstrucción 100% desde ficheros (robusta a formatos)

import os, re, json, math
from pathlib import Path
import numpy as np
import pandas as pd

# ——— Fallback por si _read_forgetting no quedó definido en la Celda 1 ———
try:
    _read_forgetting
except NameError:
    def _read_forgetting(run_dir: Path):
        def _nanmean2(a, b):
            vals = [x for x in [a, b] if x is not None and not math.isnan(_safe_float(x))]
            return (float(np.mean([_safe_float(x) for x in vals])) if vals else np.nan)
        summ = _read_json(run_dir / "forgetting_summary.json")
        if isinstance(summ, dict):
            c1_abs = _safe_float(summ.get("circuito1_forget_abs", summ.get("task_1_forget_abs")))
            c1_rel = _safe_float(summ.get("circuito1_forget_rel", summ.get("task_1_forget_rel")))
            c2_abs = _safe_float(summ.get("circuito2_forget_abs", summ.get("task_2_forget_abs")))
            c2_rel = _safe_float(summ.get("circuito2_forget_rel", summ.get("task_2_forget_rel")))
            avg_rel = _safe_float(summ.get("avg_forget_rel", summ.get("avg_forgetting_rel")))
            if math.isnan(avg_rel): avg_rel = _nanmean2(c1_rel, c2_rel)
            return {
                "circuito1_forget_abs": c1_abs, "circuito1_forget_rel": c1_rel,
                "circuito2_forget_abs": c2_abs, "circuito2_forget_rel": c2_rel,
                "avg_forget_rel": avg_rel,
            }
        js = _read_json(run_dir / "forgetting.json") or {}
        if not isinstance(js, dict): return {}
        def pick(d, *keys):
            for k in keys:
                if k in d: return d[k]
            return None
        c1_abs = _safe_float(pick(js, "circuito1_forget_abs","task_1_circuito1_forget_abs","c1_forget_abs","task_1_forget_abs"))
        c1_rel = _safe_float(pick(js, "circuito1_forget_rel","task_1_circuito1_forget_rel","c1_forget_rel","task_1_forget_rel"))
        c2_abs = _safe_float(pick(js, "circuito2_forget_abs","task_2_circuito2_forget_abs","c2_forget_abs","task_2_forget_abs"))
        c2_rel = _safe_float(pick(js, "circuito2_forget_rel","task_2_circuito2_forget_rel","c2_forget_rel","task_2_forget_rel"))
        avg_rel = _safe_float(pick(js, "avg_forget_rel","avg_forgetting_rel","mean_forget_rel","forget_rel_avg"))
        if ("circuito1" in js) and isinstance(js["circuito1"], dict):
            if math.isnan(c1_abs): c1_abs = _safe_float(js["circuito1"].get("forget_abs"))
            if math.isnan(c1_rel): c1_rel = _safe_float(js["circuito1"].get("forget_rel"))
        if ("circuito2" in js) and isinstance(js["circuito2"], dict):
            if math.isnan(c2_abs): c2_abs = _safe_float(js["circuito2"].get("forget_abs"))
            if math.isnan(c2_rel): c2_rel = _safe_float(js["circuito2"].get("forget_rel"))
        if math.isnan(avg_rel):
            vals = [x for x in [c1_rel, c2_rel] if not math.isnan(_safe_float(x))]
            avg_rel = float(np.mean([_safe_float(x) for x in vals])) if vals else np.nan
        return {
            "circuito1_forget_abs": c1_abs, "circuito1_forget_rel": c1_rel,
            "circuito2_forget_abs": c2_abs, "circuito2_forget_rel": c2_rel,
            "avg_forget_rel": avg_rel,
        }

# ——— Helpers internos ———

def _parse_basic_meta(run_dir: Path) -> dict:
    """Extrae preset, method, encoder, model, seed, T, amp, batch_size desde los artefactos.
       Prioridad: run_row.json → manifest de tarea → heurística de nombre de carpeta.
    """
    jrow = _read_json(run_dir / "run_row.json") or {}

    def gj(*ks, default=None):
        obj = jrow
        for k in ks:
            if not isinstance(obj, dict) or (k not in obj):
                return default
            obj = obj[k]
        return obj

    # Campos directos de run_row.json (top-level o dentro de meta/data/training)
    preset   = jrow.get("preset") or gj("meta","preset")
    method   = jrow.get("method") or gj("meta","method")
    encoder  = jrow.get("encoder") or gj("meta","encoder")
    model    = jrow.get("model")   or jrow.get("model_name") or gj("meta","model") or gj("meta","model_name")
    seed     = jrow.get("seed")    or gj("meta","seed")
    T        = jrow.get("T")       or gj("data","T") or gj("meta","data","T") or gj("meta","T")
    amp      = jrow.get("amp")     or gj("training","amp") or gj("meta","amp")
    batch_sz = jrow.get("batch_size") or gj("meta","batch_size")

    # Manifest de la primera tarea (por si faltan cosas)
    man1 = _read_json(run_dir / "task_1_circuito1" / "manifest.json")
    if isinstance(man1, dict):
        meta1 = man1.get("meta", {}) if isinstance(man1.get("meta", {}), dict) else {}
        if not model:        model     = meta1.get("model") or man1.get("model") or man1.get("model_name")
        if batch_sz is None: batch_sz  = meta1.get("batch_size", batch_sz)
        # Claves típicas en tu formato:
        if T is None:        T         = meta1.get("T", T)        # T vive en meta
        if amp is None:      amp       = meta1.get("amp", amp)    # amp vive en meta
        if not encoder:      encoder   = meta1.get("encoder", encoder)
        if not preset:       preset    = meta1.get("preset", preset)
        if not method:       method    = meta1.get("method", method)

    # Heurística de nombre de carpeta si faltan preset/method/encoder
    if not preset or not method or not encoder:
        name = run_dir.name
        m = re.match(r"continual_([^_]+)_([^_].*?)_(rate|latency|raw|image).*", name)
        if m:
            preset  = preset  or m.group(1)
            method  = method  or m.group(2)
            encoder = encoder or m.group(3)

    # Tipados/normalizaciones
    seed    = _safe_float(seed)
    T       = _safe_float(T)
    if isinstance(amp, str):
        amp = amp.strip().lower() in {"true","1","yes","y"}
    elif isinstance(amp, (int, float)):
        amp = bool(amp)
    elif amp is not None and not isinstance(amp, bool):
        amp = None
    batch_sz = _safe_float(batch_sz)

    return dict(
        preset=preset, method=method, encoder=encoder, model=model,
        seed=seed, T=T, amp=amp, batch_size=batch_sz
    )

def _read_per_task_perf(run_dir: Path) -> dict:
    """Devuelve dict por tarea con {'best_mae','final_mae'} normalizadas.
       Soporta per_task_perf.json (lista/dict) y per_task_perf.csv.
       Normaliza nombres de tarea (lower) y fusiona JSON+CSV para rellenar huecos.
    """
    def _norm_row_dictlike(d):
        tname = (str(d.get("task_name") or d.get("task") or d.get("name") or "")).strip().lower()
        # añadimos 'test_mae' como candidato de 'best'
        best_candidates  = ["best_mae","val_best_mae","best","mae_best","min_mae","test_mae"]
        final_candidates = ["final_mae","val_final_mae","val_last_mae","last_mae","mae_last","mae_final"]
        best  = next((d.get(k) for k in best_candidates  if k in d), None)
        final = next((d.get(k) for k in final_candidates if k in d), None)
        return tname, {"best_mae": _safe_float(best), "final_mae": _safe_float(final)}

    out = {}

    # 1) JSON
    js = _read_json(run_dir / "per_task_perf.json")
    if js is not None:
        if isinstance(js, list):
            for row in js:
                if isinstance(row, dict):
                    t, val = _norm_row_dictlike(row)
                    if t:
                        out[t] = val
        elif isinstance(js, dict):
            if all(isinstance(v, dict) for v in js.values()):
                for k, v in js.items():
                    t, val = _norm_row_dictlike({"task_name": k, **v})
                    if t:
                        out[t] = val
            else:
                try:
                    df = pd.DataFrame(js)
                    for _, row in df.iterrows():
                        t, val = _norm_row_dictlike(row.to_dict())
                        if t:
                            out[t] = val
                except Exception:
                    pass

    # 2) CSV (fusiona y rellena NaNs si los hay)
    df = _read_csv_df(run_dir / "per_task_perf.csv")
    if df is not None and not df.empty:
        for _, row in df.iterrows():
            tn = (str(row.get("task_name") or row.get("task") or row.get("name") or "")).strip().lower()
            if not tn:
                continue
            cur = out.get(tn, {"best_mae": np.nan, "final_mae": np.nan})
            if math.isnan(_safe_float(cur.get("best_mae"))):
                cur["best_mae"] = _safe_float(row.get("val_best_mae", row.get("best_mae", row.get("test_mae"))))
            if math.isnan(_safe_float(cur.get("final_mae"))):
                cur["final_mae"] = _safe_float(row.get("val_last_mae", row.get("val_final_mae", row.get("final_mae"))))
            out[tn] = cur

    return out

def _read_efficiency(run_dir: Path):
    """Lee emisiones/tiempo desde efficiency_summary.json; si falta, intenta emissions.csv y run_row.json."""
    j = _read_json(run_dir / "efficiency_summary.json") or {}
    emissions = _safe_float(j.get("emissions_kg"), default=np.nan)
    elapsed   = _safe_float(j.get("elapsed_sec"),   default=np.nan)

    if math.isnan(emissions):
        df = _read_csv_df(run_dir / "emissions.csv")
        if df is not None:
            col = "co2e_kg" if "co2e_kg" in df.columns else ("emissions_kg" if "emissions_kg" in df.columns else None)
            if col:
                emissions = float(df[col].sum())

    if math.isnan(elapsed) or math.isnan(emissions):
        rj = _read_json(run_dir / "run_row.json") or {}
        if math.isnan(elapsed):
            elapsed = _safe_float(rj.get("elapsed_sec"), default=elapsed)
        if math.isnan(emissions):
            emissions = _safe_float(rj.get("emissions_kg"), default=emissions)

    return emissions, elapsed

def _read_eval_matrix(run_dir: Path) -> pd.DataFrame | None:
    """Carga eval_matrix (csv preferente) o reconstruye desde json {'tasks':[], 'mae_matrix':[[]]}."""
    p_csv = run_dir / "eval_matrix.csv"
    if p_csv.exists():
        try:
            return pd.read_csv(p_csv)
        except Exception:
            pass

    js = _read_json(run_dir / "eval_matrix.json")
    if isinstance(js, dict) and ("tasks" in js) and ("mae_matrix" in js):
        tasks = list(js.get("tasks") or [])
        mat   = js.get("mae_matrix") or []
        # Intentamos formato de columnas como en CSV: 'task', 'after_circuito1', 'after_circuito2',...
        try:
            mat = np.array(mat, dtype=float)
            cols = ["task"] + [f"after_{t}" for t in tasks]
            data = {"task": tasks}
            for j, cname in enumerate(cols[1:]):
                colvals = [row[j] if j < len(row) else np.nan for row in mat]
                data[cname] = colvals
            return pd.DataFrame(data)
        except Exception:
            # fallback genérico
            try:
                return pd.DataFrame(js)
            except Exception:
                return None
    return None

def _compute_best_final_from_eval(eval_df: pd.DataFrame, task_token: str):
    """Obtiene BEST/FINAL mirando primero por FILAS (col 'task') y,
    si no existe, hace fallback a columnas que contengan el token."""
    if eval_df is None or eval_df.empty:
        return (np.nan, np.nan)

    # 1) Preferir formato fila: 'task' == task_token
    tcol = None
    for c in eval_df.columns:
        if str(c).strip().lower() == "task":
            tcol = c
            break
    if tcol is not None:
        mask = eval_df[tcol].astype(str).str.lower() == str(task_token).lower()
        if mask.any():
            row = eval_df.loc[mask].iloc[0]
            data_cols = [c for c in eval_df.columns if c != tcol]

            vals = pd.to_numeric(row[data_cols], errors="coerce").values.astype(float)
            finite = np.isfinite(vals)
            best = float(np.nanmin(vals)) if finite.any() else np.nan

            # FINAL = valor en la última columna temporal (p.ej., after_última_tarea)
            final = _safe_float(row[data_cols[-1]])
            return (best, final)

    # 2) Fallback: columnas que contengan el token (formatos antiguos)
    cols = [c for c in eval_df.columns if str(task_token).lower() in str(c).lower()]
    if cols:
        dfc = eval_df[cols].apply(pd.to_numeric, errors="coerce")
        vals = dfc.values.astype(float)
        finite = np.isfinite(vals)
        best = float(np.nanmin(vals)) if finite.any() else np.nan
        final = _safe_float(dfc.iloc[-1, -1])
        return (best, final)

    return (np.nan, np.nan)


def _compute_forgetting_from_eval_matrix(eval_df: pd.DataFrame, per_task: dict):
    """Olvido para c1 tras aprender c2. C2 no olvida (0)."""
    out = {}
    t1_keys = [k for k in (per_task or {}).keys() if "circuito1" in str(k).lower()]
    best_t1 = None
    if t1_keys:
        best_t1 = _safe_float((per_task.get(t1_keys[0]) or {}).get("best_mae"))
    if best_t1 is None or math.isnan(best_t1):
        best_t1, _ = _compute_best_final_from_eval(eval_df, "circuito1")

    _, final_t1_after_last = _compute_best_final_from_eval(eval_df, "circuito1")
    if best_t1 is None or math.isnan(best_t1) or final_t1_after_last is None or math.isnan(final_t1_after_last):
        return out

    forget_abs = max(0.0, final_t1_after_last - best_t1)
    forget_rel = forget_abs / max(1e-9, best_t1)
    out["circuito1_forget_abs"] = forget_abs
    out["circuito1_forget_rel"] = forget_rel
    out["circuito2_forget_abs"] = 0.0
    out["circuito2_forget_rel"] = 0.0
    out["avg_forget_rel"] = forget_rel
    return out

def _read_continual_results(run_dir: Path) -> dict:
    """Fallback final para MAEs desde continual_results.json."""
    j = _read_json(run_dir / "continual_results.json")
    if not isinstance(j, dict):
        return {}
    c1 = j.get("circuito1", {}) or {}
    c2 = j.get("circuito2", {}) or {}
    return {
        "c1_best":  _safe_float(c1.get("test_mae")),
        "c1_final": _safe_float(c1.get("after_circuito2_mae")),
        "c2_best":  _safe_float(c2.get("test_mae")),
        "c2_final": _safe_float(c2.get("test_mae")),
    }

def build_results_table_from_disk(base_out: Path) -> pd.DataFrame:
    rows = []
    run_dirs = [p for p in base_out.glob("continual_*") if p.is_dir()]
    print(f"[INFO] Escaneando {len(run_dirs)} runs en {base_out}")

    for rd in run_dirs:
        meta      = _parse_basic_meta(rd)
        per_task  = _read_per_task_perf(rd)
        eff_kg, elapsed = _read_efficiency(rd)
        forget_js = _read_forgetting(rd) or {}
        eval_df   = _read_eval_matrix(rd)

        # MAEs por tarea con fallback a eval_matrix y, si hace falta, continual_results.json
        c1_best = c1_final = c2_best = c2_final = np.nan

        t1_keys = [k for k in per_task.keys() if "circuito1" in str(k).lower()]
        if t1_keys:
            c1_best  = _safe_float(per_task[t1_keys[0]].get("best_mae"),  default=np.nan)
            c1_final = _safe_float(per_task[t1_keys[0]].get("final_mae"), default=np.nan)
        if math.isnan(c1_best) or math.isnan(c1_final):
            b, f = _compute_best_final_from_eval(eval_df, "circuito1")
            if math.isnan(c1_best):  c1_best  = b
            if math.isnan(c1_final): c1_final = f

        t2_keys = [k for k in per_task.keys() if "circuito2" in str(k).lower()]
        if t2_keys:
            c2_best  = _safe_float(per_task[t2_keys[0]].get("best_mae"),  default=np.nan)
            c2_final = _safe_float(per_task[t2_keys[0]].get("final_mae"), default=np.nan)
        if math.isnan(c2_best) or math.isnan(c2_final):
            b, f = _compute_best_final_from_eval(eval_df, "circuito2")
            if math.isnan(c2_best):  c2_best  = b
            if math.isnan(c2_final): c2_final = f

        # Fallback definitivo: continual_results.json
        if any(math.isnan(x) for x in [c1_best, c1_final, c2_best, c2_final]):
            cr = _read_continual_results(rd)
            if math.isnan(c1_best):  c1_best  = _safe_float(cr.get("c1_best"),  default=c1_best)
            if math.isnan(c1_final): c1_final = _safe_float(cr.get("c1_final"), default=c1_final)
            if math.isnan(c2_best):  c2_best  = _safe_float(cr.get("c2_best"),  default=c2_best)
            if math.isnan(c2_final): c2_final = _safe_float(cr.get("c2_final"), default=c2_final)

        # Olvido (summary/json) o cálculo desde eval_matrix
        f_c1_abs = _safe_float(forget_js.get("circuito1_forget_abs"))
        f_c1_rel = _safe_float(forget_js.get("circuito1_forget_rel"))
        f_c2_abs = _safe_float(forget_js.get("circuito2_forget_abs"))
        f_c2_rel = _safe_float(forget_js.get("circuito2_forget_rel"))
        avg_f_rel = _safe_float(forget_js.get("avg_forget_rel"))
        if all(math.isnan(x) for x in [f_c1_abs, f_c1_rel, f_c2_abs, f_c2_rel, avg_f_rel]):
            comp = _compute_forgetting_from_eval_matrix(eval_df, per_task)
            if comp:
                f_c1_abs = comp.get("circuito1_forget_abs", f_c1_abs)
                f_c1_rel = comp.get("circuito1_forget_rel", f_c1_rel)
                f_c2_abs = comp.get("circuito2_forget_abs", f_c2_abs)
                f_c2_rel = comp.get("circuito2_forget_rel", f_c2_rel)
                avg_f_rel = comp.get("avg_forget_rel", avg_f_rel)

        row = dict(
            run_dir=str(rd.relative_to(base_out)),
            preset=meta["preset"],
            method=meta["method"],
            encoder=meta["encoder"],
            model=meta["model"],
            seed=meta["seed"],
            T=meta["T"],
            batch_size=meta["batch_size"],
            amp=meta["amp"],
            emissions_kg=eff_kg,
            elapsed_sec=elapsed,
            circuito1_best_mae=c1_best,
            circuito1_final_mae=c1_final,
            circuito2_best_mae=c2_best,
            circuito2_final_mae=c2_final,
            circuito1_forget_abs=f_c1_abs,
            circuito1_forget_rel=f_c1_rel,
            circuito2_forget_abs=f_c2_abs,
            circuito2_forget_rel=f_c2_rel,
            avg_forget_rel=avg_f_rel,
        )
        rows.append(row)

    df = pd.DataFrame(rows)

    # Flags extra
    df["is_new_runner"] = df["run_dir"].apply(lambda rd: (_abs_run_dir(rd) / "run_row.json").exists() or (_abs_run_dir(rd) / "run_row.csv").exists())
    df["mtime"] = df["run_dir"].apply(run_mtime)
    df["mtime_dt"] = pd.to_datetime(df["mtime"], unit="s")
    df["method_base"] = df["method"].astype(str).apply(canonical_method)

    # Tipado numérico: EXCLUYE 'amp' (lo normalizamos como boolean más abajo)
    numeric_cols = [
        "seed","T","batch_size","emissions_kg","elapsed_sec",
        "circuito1_best_mae","circuito1_final_mae","circuito2_best_mae","circuito2_final_mae",
        "circuito1_forget_abs","circuito1_forget_rel","circuito2_forget_abs","circuito2_forget_rel","avg_forget_rel"
    ]
    for c in numeric_cols:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")

    # 'amp' como boolean (permite NA)
    if "amp" in df.columns:
        if df["amp"].dtype == bool:
            df["amp"] = df["amp"].astype("boolean")
        else:
            df["amp"] = df["amp"].map(
                lambda v: bool(v) if isinstance(v, (bool,int,float))
                else (str(v).strip().lower() in {"true","1","yes","y"} if isinstance(v, str) else pd.NA)
            ).astype("boolean")

    out_csv = SUMMARY / "results_table_fromdisk.csv"
    df.to_csv(out_csv, index=False)
    print(f"[OK] results_table_fromdisk → {out_csv} | filas:", len(df))
    return df


<a id="sec-04"></a>
## 4) Resumen de entrenamiento por tarea (manifest/metrics)

**Objetivo**  
Leer, para cada carpeta `outputs/continual_*/*task_*`, el archivo `manifest.json` (o `metrics.json`) y extraer:
- **Hiperparámetros efectivos** por tarea: `epochs`, `batch_size`, `lr`, `amp`, `seed`.  
- **Historial de pérdidas**: últimas `train_loss` y `val_loss`.

Notas:
- Si estaba activo **Early Stopping**, el número real de épocas útiles puede ser **menor** que el configurado.  
- Si una carpeta de tarea no contiene `manifest.json` ni `metrics.json`, simplemente se ignora en este resumen.  
- Este bloque **no recalcula** métricas; solo muestra lo **registrado** durante el entrenamiento.

[↑ Volver al índice](#toc)


In [ ]:
# Celda 3 — Construcción + diagnóstico NaNs

df_all = build_results_table_from_disk(OUT)
display(df_all.head(3))

nan_cols = ["circuito1_best_mae","circuito1_final_mae","circuito2_best_mae","circuito2_final_mae","avg_forget_rel"]
print("[DEBUG] NaNs globales:", {c:int(df_all[c].isna().sum()) for c in nan_cols if c in df_all.columns})


<a id="sec-05"></a>
## 5) Gráficos rápidos (MAE, olvido, emisiones)

**Objetivo**  
Ofrecer una vista rápida y comparativa:
- Barras de **`c1_mae`** por experimento (calidad en la primera tarea).  
- Barras de **olvido relativo** (`c1_forgetting_mae_rel_%`) si la columna existe y hay ≥2 tareas.  
- **NUEVO**: Barras de **emisiones totales (kg CO₂e)** por experimento y *scatter* **MAE vs emisiones** (trade-off rendimiento/sostenibilidad), usando el CSV enriquecido.

> Gráficos intencionadamente simples para inspección rápida. Para informes finales, exporta el CSV y construye figuras personalizadas.

[↑ Volver al índice](#toc)


In [ ]:
# Celda 4 — Selección y comparabilidad
def _filter_paperset(df: pd.DataFrame, require_both_tasks: bool = True) -> pd.DataFrame:
    m = df.copy()
    if PRESET_KEEP:
        m = m[m["preset"].isin(PRESET_KEEP)]
    if ENCODER_KEEP:
        m = m[m["encoder"].isin(ENCODER_KEEP)]
    if SEED_KEEP:
        m = m[m["seed"].isin(SEED_KEEP)]
    if METHODS_KEEP:
        m = m[m["method_base"].isin(METHODS_KEEP)]
    if ONLY_NEW_RUNNER:
        m = m[m["is_new_runner"] == True]
    if MTIME_FROM is not None:
        m = m[m["mtime_dt"] >= MTIME_FROM]

    # Runs con ambas tareas evaluadas (mejor y final). Evita arrastrar runs truncados.
    if require_both_tasks:
        must_cols = ["circuito1_best_mae","circuito1_final_mae","circuito2_best_mae","circuito2_final_mae"]
        for c in must_cols:
            if c in m.columns:
                m = m[~m[c].isna()]

    print(f"[DEBUG] filtros duros → inicio → {len(df)} runs | post → {len(m)} runs")
    return m

def _comparability_slice(df: pd.DataFrame, strict: bool = True):
    if df.empty:
        return df, {"estrategia":"empty", "kept":0, "dropped":0}
    # Siempre mismo modelo y mismo T
    modes_model = df["model"].dropna().unique().tolist()
    modes_T     = df["T"].dropna().unique().tolist()
    if len(modes_model) > 1:
        top_model = df["model"].value_counts().idxmax()
        df = df[df["model"] == top_model]
    if len(modes_T) > 1:
        top_T = df["T"].value_counts().idxmax()
        df = df[df["T"] == top_T]

    kept_before = len(df)
    if strict:
        # AMP mayoritario (si hay), manteniendo NaN
        if df["amp"].notna().any():
            top_amp = df["amp"].value_counts(dropna=True).idxmax()
            df = df[(df["amp"].isna()) | (df["amp"] == top_amp)]
        # batch_size mayoritario (si hay), manteniendo NaN
        if df["batch_size"].notna().any():
            top_bs = df["batch_size"].value_counts(dropna=True).idxmax()
            df = df[(df["batch_size"].isna()) | (df["batch_size"] == top_bs)]
        strategy = "strict:model+T(+amp+batch)"
    else:
        strategy = "relaxed:model+T"

    kept_after = len(df)
    return df, {"estrategia": strategy, "kept": kept_after, "dropped": kept_before - kept_after}

df_sel0 = _filter_paperset(df_all, require_both_tasks=True)
df_sel, stats = _comparability_slice(df_sel0, strict=STRICT_CFG)
if df_sel.empty and STRICT_CFG:
    print("[WARN] Comparabilidad dejó 0 runs. Relajando AMP y batch_size…")
    df_sel, stats = _comparability_slice(df_sel0, strict=False)

# Copia para informes (se puede filtrar naive aquí si se desea solo de cara a tablas/plots)
df_sel["batch_size_filled"] = df_sel["batch_size"].copy()
df_report = df_sel.copy()
if IGNORE_NAIVE_IN_REPORTS and "method_base" in df_report.columns:
    df_report = df_report[df_report["method_base"] != "naive"]

display(df_sel.head(10))
print(f"[OK] Selección final: {len(df_sel)} runs | estrategia: {stats}")

out_csv = SUMMARY / "selection_table.csv"
df_sel.to_csv(out_csv, index=False)
print("[OK] Selección →", out_csv)

# --- NUEVO: guarda la ruta en _last.json
_last_update(selection=out_csv)


<a id="sec-06"></a>
## 6) Inspección de un experimento concreto

**Objetivo**  
Abrir el `continual_results.json` del **experimento más reciente** (o el que indiques manualmente) y mostrar su contenido completo (diccionario).
- Útil para revisar **todas** las claves guardadas por el *runner* y verificar cálculos.  
- Si no existen carpetas `continual_*`, se informa en consola y no se imprime nada.

> Pista: revisa las secciones por **tarea** y los bloques `after_*` que reflejan el rendimiento **tras** aprender nuevas tareas (base del cálculo de olvido).

[↑ Volver al índice](#toc)


In [ ]:
# Celda 5 — Ranking compuesto + winners por método (versión robusta)
def _ensure_mae_columns(df: pd.DataFrame) -> pd.DataFrame:
    """Crea alias entre c2_final_mae y circuito2_final_mae si falta alguno."""
    df = df.copy()
    has_c2    = "c2_final_mae" in df.columns
    has_circ2 = "circuito2_final_mae" in df.columns
    if has_c2 and not has_circ2:
        df["circuito2_final_mae"] = df["c2_final_mae"]
    elif has_circ2 and not has_c2:
        df["c2_final_mae"] = df["circuito2_final_mae"]
    return df

def _minmax_norm(s: pd.Series) -> pd.Series:
    s = pd.to_numeric(s, errors="coerce")
    if s.size == 0:
        return s
    vals = s.values.astype(float)
    finite = np.isfinite(vals)
    if finite.sum() == 0:
        return pd.Series(np.full(len(s), np.nan), index=s.index)
    lo, hi = float(np.nanmin(vals)), float(np.nanmax(vals))
    if not np.isfinite(lo) or not np.isfinite(hi) or hi <= lo:
        out = np.zeros(len(s), dtype=float)
        out[~finite] = np.nan
        return pd.Series(out, index=s.index)
    return (s - lo) / (hi - lo)

# 1) Copia y compatibilidad de columnas
df_rank = _ensure_mae_columns(df_report)

# Columnas clave
MAE_COL  = "circuito2_final_mae"
FORG_COL = "avg_forget_rel"

# 2) Si está vacío, escribe CSV vacío y sal con aviso
if df_rank.empty:
    winners = df_rank.copy()
    out_csv = SUMMARY / "winners_per_method.csv"
    winners.to_csv(out_csv, index=False)
    print("[WARN] df_rank está vacío tras filtros. Winners vacío →", out_csv)
    display(winners)  # DF vacío
else:
    mae_norm  = _minmax_norm(df_rank[MAE_COL]  if MAE_COL  in df_rank.columns else pd.Series([], dtype=float))
    forg_norm = _minmax_norm(df_rank[FORG_COL] if FORG_COL in df_rank.columns else pd.Series([], dtype=float))

    score_comp = ALPHA_COMPOSITE * mae_norm + (1 - ALPHA_COMPOSITE) * forg_norm
    score_eff  = score_comp.copy()
    if len(score_eff) != 0:
        score_eff[forg_norm.isna()] = mae_norm[forg_norm.isna()]  # fallback

    df_rank["score_comp"] = score_comp
    df_rank["score_eff"]  = score_eff

    # Agrupación: por método base o por método completo (separa composites)
    group_col = "method" if GROUP_BY_FULL_METHOD else "method_base"
    if group_col not in df_rank.columns:
        df_rank[group_col] = df_rank["method"] if "method" in df_rank.columns else "unknown"

    # Winners
    if df_rank["score_eff"].notna().any():
        winners = (
            df_rank
            .sort_values([group_col, "score_eff", MAE_COL], ascending=[True, True, True])
            .groupby(group_col, as_index=False)
            .head(1)
            .sort_values("score_eff", ascending=True)
        )
    else:
        winners = (
            df_rank
            .sort_values([group_col, MAE_COL], ascending=[True, True])
            .groupby(group_col, as_index=False)
            .head(1)
        )

    # Export
    out_csv = SUMMARY / f"winners_per_{'fullmethod' if GROUP_BY_FULL_METHOD else 'methodbase'}.csv"
    winners.to_csv(out_csv, index=False)

    n_rows = len(df_rank)
    n_groups = df_rank[group_col].nunique()
    print(f"[OK] Winners → {out_csv} | candidatos={n_rows} | grupos={n_groups}")
    cols_show = [c for c in ["run_dir", group_col, "seed", MAE_COL, FORG_COL, "emissions_kg", "score_eff"] if c in winners.columns]
    display(winners[cols_show] if cols_show else winners)

# --- NUEVO: guarda la ruta de winners
_last_update(winners=out_csv)
# Mantén "winners" en el entorno por si otras celdas lo usan


<a id="sec-07"></a>
## 7) Inspección detallada: CodeCarbon + Telemetry (último run)

**Objetivo**  
Mostrar los **últimos registros** de `emissions.csv` (CodeCarbon) y la **última entrada** de `telemetry.jsonl` para el run más reciente:
- Tabla con las **últimas filas** de `emissions.csv` (para ver el cierre del tracker).  
- Resumen rápido: `emissions_kg`, `energy_kwh`, `duration_s`.  
- Último evento de `telemetry.jsonl` (suele incluir `elapsed_sec` y, si estaba disponible, `emissions_kg`).

> Útil para validar que el tracker **cerró bien** y que los tiempos/emisiones concuerdan con lo registrado por el runner.

[↑ Volver al índice](#toc)


In [ ]:
# Celda 5.1 — Deltas relativos vs baseline (para narrativa TFM)
def _row_key_for_match(r):
    """Clave de emparejamiento para baseline estricto."""
    return (
        r.get("preset"), r.get("encoder"), r.get("model"), r.get("T"),
        r.get("amp"), r.get("batch_size"), r.get("seed")
    )

def compute_relative_to_baseline(df_in: pd.DataFrame, baseline="naive", strict=True):
    if not baseline:
        return None
    if isinstance(baseline, str) and baseline.lower() == "auto":
        bases = df_in["method_base"].dropna().str.lower().unique().tolist()
        baseline = "ewc" if "ewc" in bases else ("naive" if "naive" in bases else None)
        if not baseline:
            return None
    dfb = df_in.copy()
    dfb["__key__"] = dfb.apply(
        lambda r: (r.get("preset"), r.get("encoder"), r.get("model"), r.get("T"),
                   r.get("amp"), r.get("batch_size"), r.get("seed")), axis=1)

    # Para cada clave, escoge el run baseline con menor MAE y usa sus dos métricas
    base_rows = []
    for key, g in dfb[dfb["method_base"].astype(str).str.lower() == baseline.lower()].groupby("__key__"):
        if g.empty:             continue
        idx = g["circuito2_final_mae"].astype(float).idxmin()
        row = g.loc[idx]
        base_rows.append({
            "__key__": key,
            "baseline_mae": float(row["circuito2_final_mae"]),
            "baseline_forget": float(row["avg_forget_rel"])
        })
    base_tbl = pd.DataFrame(base_rows)

    out = dfb.merge(base_tbl, on="__key__", how="left")
    out["delta_mae_vs_base"]    = out["circuito2_final_mae"] - out["baseline_mae"]
    out["delta_forget_vs_base"] = out["avg_forget_rel"]      - out["baseline_forget"]
    return out

if RELATIVE_BASELINE:
    df_rel = compute_relative_to_baseline(df_sel, baseline=RELATIVE_BASELINE, strict=BASELINE_MATCH_STRICT)
    if df_rel is not None:
        out_csv = SUMMARY / f"relative_to_{RELATIVE_BASELINE}.csv"
        df_rel.to_csv(out_csv, index=False)
        print(f"[OK] Relative-to-baseline → {out_csv}")

        # --- NUEVO: guarda la ruta en _last.json
        _last_update(relative_baseline=RELATIVE_BASELINE, relative_csv=out_csv)

        # Top-10 mejoras vs baseline (orden: delta_mae luego delta_forget)
        mask_has_base = (~df_rel["baseline_mae"].isna()) & (~df_rel["baseline_forget"].isna())
        top_improve = (df_rel[mask_has_base]
                       .sort_values(["delta_mae_vs_base","delta_forget_vs_base"], ascending=[True, True])
                       .head(10))
        cols_show = ["run_dir","method","method_base","circuito2_final_mae","avg_forget_rel",
                     "baseline_mae","baseline_forget","delta_mae_vs_base","delta_forget_vs_base"]
        display(top_improve[cols_show])


<a id="sec-08"></a>
## 8) Agregados por preset/método con emisiones

**Objetivo**  
Calcular y mostrar agregados **por `preset`** y **por `method`** a partir de `runs_with_emissions.csv`:
- `runs` (conteo), `mean_c1_mae`, `sum_emissions_kg`, `mean_emissions_kg`.

> Te da una visión global del **trade-off** por configuración: calidad media y huella total/media.  
> Si no existe el CSV enriquecido, ejecuta antes la **Sección 3**.

[↑ Volver al índice](#toc)


In [ ]:
# Celda 5.2 — Porcentajes vs EWC y scorecards por método (sin dependencias extra)
import pandas as pd, numpy as np
from pathlib import Path

# Asegura que tenemos df_rel (de la Celda 5.1)
try:
    df_rel
except NameError:
    df_rel = compute_relative_to_baseline(df_sel, baseline="ewc", strict=True)

# 1) % mejora en MAE y cambio de olvido en puntos porcentuales (pp)
rel = df_rel.copy()
rel["mae_improv_pct"] = 100.0 * (rel["baseline_mae"] - rel["circuito2_final_mae"]) / rel["baseline_mae"]
rel["forget_delta_pp"] = 100.0 * (rel["avg_forget_rel"] - rel["baseline_forget"])

# 2) “Winners” por método ya calculado en Celda 5; si no existe, lo recomputamos rápido
try:
    winners
except NameError:
    tmp = df_rel.copy()
    # Normaliza columnas si faltan
    if "c2_final_mae" not in tmp.columns and "circuito2_final_mae" in tmp.columns:
        tmp["c2_final_mae"] = tmp["circuito2_final_mae"]
    # Score simple si no hay score_eff
    if "score_eff" not in tmp.columns:
        tmp["score_eff"] = np.nan
    group_col = "method_base" if "method_base" in tmp.columns else "method"
    winners = (tmp.sort_values([group_col, "c2_final_mae"], ascending=[True, True])
                   .groupby(group_col, as_index=False).head(1))

# 3) “Scorecard” por método: junta winners con porcentajes vs baseline
key_cols = ["run_dir","method","method_base","preset","encoder","model","seed","T","batch_size","amp"]
cols_metrics = ["circuito2_final_mae","avg_forget_rel","baseline_mae","baseline_forget",
                "mae_improv_pct","forget_delta_pp","emissions_kg","elapsed_sec"]
scorecards = winners[key_cols].merge(rel[key_cols + cols_metrics], on=key_cols, how="left")

# 4) Exporta CSV legible para la memoria
out_dir = SUMMARY
scorecards_csv = out_dir / "scorecards_por_metodo_vs_ewc.csv"
scorecards.to_csv(scorecards_csv, index=False)
print("[OK] Scorecards →", scorecards_csv)

# 5) (Opcional) si tus ángulos están normalizados [-1,1], dar MAE en grados (±25° típico)
STEER_REP = os.environ.get("STEER_REP", "unknown")  # "norm1" si [-1,1], "deg" si ya son grados
DEG_RANGE = float(os.environ.get("DEG_RANGE", "25"))  # cambia si tu rango real es ±X grados

def _mae_to_deg(mae):
    try:
        v = float(mae)
        if STEER_REP.lower() == "norm1":
            return v * DEG_RANGE
        return v  # ya en grados o desconocido
    except Exception:
        return np.nan

scorecards["c2_final_mae_deg"] = scorecards["circuito2_final_mae"].apply(_mae_to_deg)
scorecards.to_csv(scorecards_csv, index=False)  # re-escribe con la columna extra
print("[OK] Añadido c2_final_mae_deg (si procede) →", scorecards_csv)

# 6) Vista rápida ordenada por “mejor” (↑% mejora MAE, ↓Δ olvido)
show_cols = ["method_base","method","circuito2_final_mae","c2_final_mae_deg","avg_forget_rel",
             "baseline_mae","baseline_forget","mae_improv_pct","forget_delta_pp","emissions_kg"]
disp = (scorecards.sort_values(["mae_improv_pct","forget_delta_pp"], ascending=[False, True]))[show_cols]
display(disp)


In [ ]:
# Celda 6 — Top-N global y Pareto (versión robusta)
# helpers (por si no vienes de la Celda 5)
try:
    _ = _ensure_mae_columns
except NameError:
    def _ensure_mae_columns(df: pd.DataFrame) -> pd.DataFrame:
        df = df.copy()
        has_c2 = "c2_final_mae" in df.columns
        has_circ2 = "circuito2_final_mae" in df.columns
        if has_c2 and not has_circ2:
            df["circuito2_final_mae"] = df["c2_final_mae"]
        elif has_circ2 and not has_c2:
            df["c2_final_mae"] = df["circuito2_final_mae"]
        return df

df_rank = _ensure_mae_columns(df_report if "df_report" in globals() else df_sel.copy())
if "score_eff" not in df_rank.columns:
    df_rank = df_rank.copy()
    df_rank["score_eff"] = np.nan
if "method_base" not in df_rank.columns:
    df_rank["method_base"] = df_rank["method"] if "method" in df_rank.columns else "unknown"

MAE_COL = "circuito2_final_mae" if "circuito2_final_mae" in df_rank.columns else ("c2_final_mae" if "c2_final_mae" in df_rank.columns else "circuito2_final_mae")

# Top-N compuesto
TOPN = 6
if df_rank.empty:
    topn = df_rank.copy()
    print("[WARN] df_rank está vacío; Top-N vacío.")
else:
    cols_sort = [c for c in ["score_eff", MAE_COL] if c in df_rank.columns]
    if not cols_sort:
        topn = df_rank.head(TOPN)
    else:
        topn = df_rank.sort_values(cols_sort, ascending=[True] * len(cols_sort)).head(TOPN)

out_csv = SUMMARY / "top6_composite.csv"
topn.to_csv(out_csv, index=False)
print("[OK] Top-6 →", out_csv)

# --- NUEVO: guarda la ruta
_last_update(top6=out_csv)

cols_show = [c for c in ["run_dir","method_base","seed",MAE_COL,"avg_forget_rel","emissions_kg","score_eff"] if c in topn.columns]
display(topn[cols_show] if cols_show else topn)

# Pareto (minimiza MAE y olvido); NaN -> +inf para no dominar
tmp = _ensure_mae_columns(df_report.copy())
if "method_base" not in tmp.columns:
    tmp["method_base"] = tmp["method"] if "method" in tmp.columns else "unknown"
if MAE_COL not in tmp.columns:
    tmp[MAE_COL] = np.nan

tmp["_olvido_for_pareto"] = pd.to_numeric(tmp.get("avg_forget_rel", np.nan), errors="coerce")
tmp.loc[tmp["_olvido_for_pareto"].isna(), "_olvido_for_pareto"] = np.inf
tmp["_mae_for_pareto"] = pd.to_numeric(tmp.get(MAE_COL, np.nan), errors="coerce")
tmp.loc[tmp["_mae_for_pareto"].isna(), "_mae_for_pareto"] = np.inf

pareto_idx = []
vals = tmp[["_mae_for_pareto", "_olvido_for_pareto"]].values
for i in range(len(tmp)):
    mae_i, forg_i = vals[i]
    dominated = False
    for j in range(len(tmp)):
        if j == i: continue
        mae_j, forg_j = vals[j]
        if (mae_j <= mae_i) and (forg_j <= forg_i) and ((mae_j < mae_i) or (forg_j < forg_i)):
            dominated = True
            break
    if not dominated:
        pareto_idx.append(i)

df_pareto = tmp.iloc[pareto_idx].drop(columns=["_olvido_for_pareto", "_mae_for_pareto"])
out_csv = SUMMARY / "pareto.csv"
df_pareto.to_csv(out_csv, index=False)
print("[OK] Pareto →", out_csv)

# --- NUEVO: guarda la ruta
_last_update(pareto=out_csv)

cols_show = [c for c in ["run_dir","method_base",MAE_COL,"avg_forget_rel"] if c in df_pareto.columns]
display(df_pareto[cols_show] if cols_show else df_pareto)


In [ ]:
# Celda 7 — Scatter MAE vs Olvido
import matplotlib.pyplot as plt

def _scatter(df, title, out_png: Path):
    x = pd.to_numeric(df["avg_forget_rel"], errors="coerce")
    y = pd.to_numeric(df["circuito2_final_mae"], errors="coerce")
    plt.figure(figsize=(7,5))
    plt.scatter(x, y)  # sin estilos ni colores específicos
    plt.xlabel("avg_forget_rel (↓ mejor)")
    plt.ylabel("circuito2_final_mae (↓ mejor)")
    plt.title(title)
    plt.grid(True, which="both", linestyle="--", alpha=0.3)
    plt.tight_layout()
    plt.savefig(out_png, dpi=160)
    plt.close()

_scatter(df_sel, "Todos", SUMMARY / "scatter_all.png")
_scatter(winners if "winners" in globals() else df_sel.head(0), "Winners por método", SUMMARY / "scatter_winners.png")
print("[OK] Scatter →", SUMMARY / "scatter_all.png")
print("[OK] Scatter →", SUMMARY / "scatter_winners.png")

# --- NUEVO: guarda las rutas
_last_update(scatter_all=SUMMARY / "scatter_all.png",
             scatter_winners=SUMMARY / "scatter_winners.png")


In [ ]:
# Celda 8 — Trazabilidad por run

def explain_sources(run_row):
    rd = _abs_run_dir(run_row["run_dir"])
    exists = {
        "per_task_perf.json": (rd / "per_task_perf.json").exists(),
        "per_task_perf.csv":  (rd / "per_task_perf.csv").exists(),
        "forgetting.json":    (rd / "forgetting.json").exists(),
        "eval_matrix.csv":    (rd / "eval_matrix.csv").exists(),
        "eval_matrix.json":   (rd / "eval_matrix.json").exists(),
        "efficiency_summary.json": (rd / "efficiency_summary.json").exists(),
        "emissions.csv":      (rd / "emissions.csv").exists(),
        "run_row.json":       (rd / "run_row.json").exists(),
        "task_1_circuito1/manifest.json": (rd / "task_1_circuito1/manifest.json").exists(),
    }
    return exists

print("## NaN en selección ##")
cols_check = ["circuito1_best_mae","circuito1_final_mae","circuito2_final_mae","avg_forget_rel"]
mask_nans = df_sel[cols_check].isna().any(axis=1)
df_nans = df_sel[mask_nans].copy()
print(f"[INFO] {len(df_nans)} runs con NaN en {cols_check}:")
display(df_nans[["run_dir","method_base","preset","encoder","seed","T","amp","batch_size"] + cols_check] if len(df_nans) else df_nans)

for _, r in df_sel.iterrows():
    ex = explain_sources(r)
    ok_flags = " | ".join([f"{k} → {'OK' if v else 'NO'}" for k,v in ex.items()])
    print(f"— {r['run_dir']}\n    {ok_flags}")


In [ ]:
# Celda 9 — Sanidad naive vs CL

DF = df_sel.copy()
agg = (
    DF.groupby("method_base", dropna=False)
      .agg(forget_median=("avg_forget_rel","median"),
           forget_mean=("avg_forget_rel","mean"),
           mae_mean=("circuito2_final_mae","mean"),
           runs=("run_dir","count"))
      .reset_index()
)
display(agg)

naive_med = agg.loc[agg["method_base"]=="naive","forget_median"].values
cl_med    = agg.loc[agg["method_base"]!="naive","forget_median"].median()
print(f"[CHECK] naive_median_forget={naive_med[0] if len(naive_med) else np.nan} vs CL_median={cl_med}")
if len(naive_med) and (np.isnan(naive_med[0]) or (not np.isnan(cl_med) and naive_med[0] <= cl_med)):
    print("[WARN] Naive NO olvida más que CL. Revisa eval_matrix/forgetting.json o implementación.")



In [ ]:
# Celda 9b — Sanity-check EWC (parse logs en carpetas de run)

import glob

def _find_log_files(run_dir: Path):
    pats = ["*.log", "stdout*.txt", "train*.log"]
    files = []
    for p in pats:
        files += list(run_dir.glob(p))
        files += list((run_dir / "task_1_circuito1").glob(p))
        files += list((run_dir / "task_2_circuito2").glob(p))
    return files

def _parse_ewc_lines(text: str):
    # Busca líneas del estilo: [EWC] base=... | pen=... | pen/base=...
    bases, pens, ratios = [], [], []
    for line in text.splitlines():
        if "[EWC]" in line:
            # intenta extraer números
            try:
                # formato flexible
                # ej: [EWC] base=0.007436 | pen=0 | pen/base=0.000
                m_base = re.search(r"base\s*=\s*([0-9.eE+-]+)", line)
                m_pen  = re.search(r"pen\s*=\s*([0-9.eE+-]+)", line)
                m_rat  = re.search(r"pen/base\s*=\s*([0-9.eE+-]+)", line)
                if m_base: bases.append(float(m_base.group(1)))
                if m_pen:  pens.append(float(m_pen.group(1)))
                if m_rat:  ratios.append(float(m_rat.group(1)))
            except Exception:
                pass
    return bases, pens, ratios

rows = []
for _, r in df_sel.iterrows():
    if not str(r.get("method","")).lower().startswith("ewc") and "ewc" not in str(r.get("method","")).lower():
        continue
    rd = _abs_run_dir(r["run_dir"])
    logs = _find_log_files(rd)
    all_bases, all_pens, all_ratios = [], [], []
    for lf in logs:
        try:
            txt = lf.read_text(encoding="utf-8", errors="ignore")
            b, p, q = _parse_ewc_lines(txt)
            all_bases += b; all_pens += p; all_ratios += q
        except Exception:
            pass
    rows.append({
        "run_dir": r["run_dir"],
        "has_logs": len(logs) > 0,
        "pen_gt0_count": int(sum(1 for x in all_pens if (isinstance(x,float) and x > 0))),
        "pen_entries": len(all_pens),
        "ratio_gt0_count": int(sum(1 for x in all_ratios if (isinstance(x,float) and x > 0))),
        "ratio_entries": len(all_ratios),
    })

df_ewc_sanity = pd.DataFrame(rows)
out_csv = SUMMARY / "ewc_sanity.csv"
df_ewc_sanity.to_csv(out_csv, index=False)
print("[OK] EWC sanity →", out_csv)
display(df_ewc_sanity)


In [ ]:
# Celda 10 — Export final
print("[OK] Artifacts en:", SUMMARY)
for p in sorted(SUMMARY.glob("*.csv")) + sorted(SUMMARY.glob("*.png")):
    print(" -", p.name)

# --- NUEVO: apunta dónde quedó el manifest de “último”
print("[LAST] Manifiesto final:", OUT / "summary" / "_last.json")


In [ ]:
# Celda X — Top-5 por método + parámetros (lee rutas desde _last.json y exporta en SUMMARY)

from pathlib import Path
import os, json, re
import pandas as pd
import numpy as np

# --- Fallbacks por si no vienes de la Celda 0
try:
    OUT
except NameError:
    OUT = Path(os.environ.get("OUT_DIR", "/home/cesar/proyectos/TFM_SNN/outputs")).resolve()

try:
    _last_update
except NameError:
    def _last_update(**kwargs):  # no-op si no está definida
        pass

LAST_PATH = OUT / "summary" / "_last.json"
LATEST = OUT / "summary" / "latest"

def _pick_latest_selection():
    # 1) _last.json
    try:
        last = json.loads(LAST_PATH.read_text(encoding="utf-8"))
        p = Path(last.get("selection", ""))
        if p.exists():
            return p
    except Exception:
        pass
    # 2) SUMMARY actual
    try:
        if "SUMMARY" in globals():
            p = Path(SUMMARY) / "selection_table.csv"
            if p.exists():
                return p
    except Exception:
        pass
    # 3) symlink latest
    try:
        p = LATEST / "selection_table.csv"
        if p.exists():
            return p
    except Exception:
        pass
    # 4) glob: el más reciente
    cands = list(OUT.glob("summary/*/selection_table.csv"))
    if not cands:
        raise FileNotFoundError("No se encontró ningún selection_table.csv en outputs/summary/**/")
    cands.sort(key=lambda x: x.stat().st_mtime, reverse=True)
    return cands[0]

# 1) Carga selección
sel_csv = _pick_latest_selection()
df = pd.read_csv(sel_csv)
print(f"[LOAD] selection_table.csv → {sel_csv}")

# 2) Normaliza columnas clave
def _canonical_method(m: str) -> str:
    s = (m or "").lower()
    if s.startswith("ewc"): return "ewc"
    if s.startswith("as-snn"): return "as-snn"
    if s.startswith("sa-snn"): return "sa-snn"
    if s.startswith("sca-snn"): return "sca-snn"
    if s.startswith("rehearsal"): return "rehearsal"
    if s.startswith("naive"): return "naive"
    return s or "unknown"

if "method_base" not in df.columns or df["method_base"].isna().all():
    df["method_base"] = df["method"].astype(str).apply(_canonical_method)
else:
    # rellena NA con prefijo del method o con canonical
    mask_na = df["method_base"].isna()
    if mask_na.any():
        df.loc[mask_na, "method_base"] = (
            df.loc[mask_na, "method"].astype(str).apply(_canonical_method)
        )

# 3) Parser de hiperparámetros desde 'method' (robusto a variaciones de nombres)
_float = lambda s: (None if s is None else float(str(s).replace("_", "").replace("p", ".")))

def parse_params(m):
    s = str(m or "").lower()
    out = {}

    # -------- AS-SNN --------
    # as-snn_gr_<float>_lam_<float>_att_f<d>   + flags: scale_on, ema\d+, _l1/_l2
    m_as = re.search(r"as-snn(?:(?:_gr[_-]?([\d._e+-]+)))?(?:_lam[_-]?([\d._e+-]+))?(?:_att[_-]?f(\d))?", s)
    if m_as:
        gr, lam, att = m_as.groups()
        if gr:  out["gr"] = _float(gr)
        if lam: out["lam"] = lam  # mantenemos texto por si es 3e+08
        if att: out["attach"] = f"f{att}"
    if "scale_on" in s: out["scale_on"] = 1
    m_ema = re.search(r"ema0?(\d+)", s)
    if m_ema: out["ema"] = int(m_ema.group(1))
    if "_l1" in s: out["penalty"] = ("l1" if "penalty" not in out else out["penalty"] + "/l1")
    if "_l2" in s: out["penalty"] = ("l2" if "penalty" not in out else out["penalty"] + "/l2")

    # -------- SA-SNN --------
    # sa-snn_k<d>_tau<d>_vt<1p33|1.33>_p<nnn|2m> + reset1 + flat[01]
    m_sa = re.search(r"sa(?:-snn)?(?:.*?_k(\d+))?(?:_tau(\d+))?(?:_vt([0-9p\._+-]+))?(?:_p([\d_]+|(\d+)m))?", s)
    if m_sa:
        k, tau, vt, p_raw, p_m = m_sa.groups()
        if k:   out["k"] = int(k)
        if tau: out["tau"] = int(tau)
        if vt:  out["vt"] = float(str(vt).replace("_","").replace("p","."))
        if p_m: out["p"] = int(p_m) * 1_000_000
        elif p_raw:
            out["p"] = int(str(p_raw).replace("_",""))
    if "reset1" in s: out["reset"] = 1
    if "flat1"  in s: out["flat"]  = 1
    elif "flat0" in s: out["flat"] = 0

    # -------- SCA-SNN --------
    # sca-snn_bins<d>_beta<flt>_bias<flt>_temp<flt>_ab<d+>
    m_sca = re.search(r"sca(?:-snn)?_bins(\d+).*?beta([-\d\.e+]+).*?bias([-\d\.e+]+).*?temp([-\d\.e+]+).*?(ab\d+)", s)
    if m_sca:
        bins, beta, bias, temp, ab = m_sca.groups()
        out.update({
            "bins": int(bins),
            "beta": float(beta),
            "bias": float(bias),
            "temp": float(temp),
            "attach_block": ab
        })

    # -------- Rehearsal --------
    # rehearsal_buf_<int>_rr_<0-100 | 0.x>
    m_reh = re.search(r"rehearsal.*?_buf[_-]?(\d+).*?_rr[_-]?([0-9\.]+)", s)
    if m_reh:
        buf, rr = m_reh.groups()
        rr_f = float(rr)
        rr_f = rr_f/100.0 if rr_f > 1.0 else rr_f
        out.update({"buffer": int(buf), "rr": rr_f})

    # -------- EWC --------
    # ewc_*_lam_<3e+08>  + fisher batches: fb\d+ o 'f\d' (evita confundir con attach)
    m_ewc = re.search(r"\bewc\b.*?_lam[_-]?([\de\+\-\.]+)", s)
    if m_ewc:
        out["lambda"] = m_ewc.group(1)
    m_fb = re.search(r"\bfb(\d+)\b", s) or re.search(r"\bf(\d+)\b", s)
    if "ewc" in s and m_fb:
        out["fisher_batches"] = int(m_fb.group(1))

    # -------- genérico: attach fX si aparece suelto y no viene de AS-SNN
    if "attach" not in out:
        m_att = re.search(r"\bf(\d)\b", s)
        if m_att:
            out["attach"] = f"f{m_att.group(1)}"

    return out

params = df["method"].apply(parse_params).apply(pd.Series)
dfp = pd.concat([df, params], axis=1)

# 4) Métrica objetivo y orden
for col_src, col_dst in [("circuito2_final_mae","c2_final_mae"),
                         ("avg_forget_rel","forget"),
                         ("emissions_kg","co2")]:
    if col_src in dfp.columns:
        dfp[col_dst] = pd.to_numeric(dfp[col_src], errors="coerce")
    else:
        dfp[col_dst] = np.nan

# 5) Top-5 por method_base
tops = []
for mb, g in dfp.groupby("method_base", dropna=False):
    g = g.sort_values(["c2_final_mae","forget","co2"], ascending=[True, True, True]).head(5)
    # columnas de hiperparámetros presentes dinámicamente
    param_cols = [c for c in [
        "gr","lam","attach","k","tau","vt","p","bins","beta","bias","temp",
        "attach_block","flat","buffer","rr","lambda","fisher_batches","scale_on","ema","penalty","reset"
    ] if c in g.columns]
    keep_cols = ["method_base","method","c2_final_mae","forget","co2"] + param_cols
    tops.append(g[keep_cols])

if tops:
    top5 = pd.concat(tops, ignore_index=True)
else:
    top5 = dfp.head(0)

# 6) Exporta en el SUMMARY correspondiente a la selección
export_dir = sel_csv.parent   # mismo SUMMARY donde está selection_table.csv
out_csv = export_dir / "top5_por_metodo.csv"
out_tex = export_dir / "top5_por_metodo.tex"

top5.to_csv(out_csv, index=False)

with open(out_tex, "w", encoding="utf-8") as f:
    for mb, g in top5.groupby("method_base", dropna=False):
        # ordena columnas: primero métricas, luego params
        base_cols = ["method","c2_final_mae","forget","co2"]
        other_cols = [c for c in g.columns if c not in (["method_base"] + base_cols)]
        gg = g[base_cols + other_cols]
        f.write(gg.to_latex(index=False, float_format="%.6f"))

print(f"[OK] → {out_csv.name} / {out_tex.name} en {export_dir}")

# registra rutas para reutilización
_last_update(top5_csv=out_csv, top5_tex=out_tex)


In [ ]:
# Celda A — Cargar y mostrar el Top-5 por método (y ruta de dónde salió)

from pathlib import Path
import json, os
import pandas as pd

OUT = Path(os.environ.get("OUT_DIR", "/home/cesar/proyectos/TFM_SNN/outputs")).resolve()
LAST = OUT / "summary" / "_last.json"

def _pick_top5_path():
    # 1) _last.json si existe
    try:
        j = json.loads(LAST.read_text(encoding="utf-8"))
        p = Path(j.get("top5_csv",""))
        if p.exists(): 
            return p
        # fallback a la carpeta del selection si solo tenemos eso
        p2 = Path(j.get("selection","")).parent / "top5_por_metodo.csv"
        if p2.exists():
            return p2
    except Exception:
        pass
    # 2) symlink 'latest'
    p = OUT / "summary" / "latest" / "top5_por_metodo.csv"
    if p.exists(): 
        return p
    # 3) glob más reciente
    cands = sorted(OUT.glob("summary/*/top5_por_metodo.csv"), key=lambda x: x.stat().st_mtime, reverse=True)
    if not cands:
        raise FileNotFoundError("No encuentro top5_por_metodo.csv; ejecuta antes la celda de Top-5.")
    return cands[0]

top5_csv = _pick_top5_path()
top5 = pd.read_csv(top5_csv)
print(f"[OK] Cargado Top-5 → {top5_csv}")
display(top5)

# Vista separada por método (para inspección rápida en el notebook)
for mb, g in top5.groupby("method_base", dropna=False):
    print(f"\n### {mb} — {len(g)} configs")
    display(g)
